In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sklearn
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
# --- Read shapefile (point file is made from centerline extracting values every 5 meter using ESRI GIS)
path = r'path\to\centerline2018_point.shp'
df = gpd.read_file(path)

In [ ]:
# Build function to resample data to 100 samples and normalize y-values to x
def pathshape(reference_value):
  data = df.loc[df['reference'] == reference_value]
  # input data (extract point from line every 5 meter)
  line = data['Z']
  n = (len(data)*5)-5
  line = resample(line, n_samples=100)
  line = (line-line.min())/n*100
  line = line.sort_index()
  arr = line.to_numpy()

  return arr

# Input data  
result_array = np.empty((0, 101))
rg = range(0, 18658, 1)

# Loop through all avalanches using function pathshape(reference_value)
for i in rg:
  try:
    result = pathshape(i)
    result = np.sort(result)
    # Flip slopes if they don't go from high values to low values
    if result[25] < result[75]:
      result = np.flip(result)
    
    result = np.append(result, [i])
    result_array = np.append(result_array, [result], axis=0)
  except Exception:
    pass

In [ ]:
out_path = r'path\to\array.csv'
np.savetxt(out_path, result_array, delimiter=",")